In [75]:
import pandas as pd
import numpy as np
import stopwords
import nltk
from nltk.corpus import stopwords
import re
from transformers import AutoTokenizer, AutoModel
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [77]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


data = pd.read_csv("itog_data.csv")
data_eval = pd.read_csv("train_data.csv")

# data['combined_questions'] = data['question_1'] + ' ' + data['question_2'] + ' ' + data['question_3'] + ' ' + data['question_4'] + ' ' + data['question_5']


In [78]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [79]:
sw = stopwords.words('russian')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^а-яА-Я?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

In [80]:
data['question_1'] = data['question_1'].apply(lambda x: clean_text(x))
data['question_2'] = data['question_2'].apply(lambda x: clean_text(x))
data['question_3'] = data['question_3'].apply(lambda x: clean_text(x))
data['question_4'] = data['question_4'].apply(lambda x: clean_text(x))
data['question_5'] = data['question_5'].apply(lambda x: clean_text(x))


data_eval['question_1'] = data_eval['question_1'].apply(lambda x: clean_text(x))
data_eval['question_2'] = data_eval['question_2'].apply(lambda x: clean_text(x))
data_eval['question_3'] = data_eval['question_3'].apply(lambda x: clean_text(x))
data_eval['question_4'] = data_eval['question_4'].apply(lambda x: clean_text(x))
data_eval['question_5'] = data_eval['question_5'].apply(lambda x: clean_text(x))


In [81]:
data

,Unnamed: 0,timestamp,question_1,question_2,question_3,question_4,question_5,is_relevant,object,is_positive
0,0,2024-04-16 09:30:00,основы программирования,преподаватель смог заинтересовать,неинтересно,"стоит пытаться улучшать, просто отменить",жуть,0,0,0.0
1,1,2024-04-17 14:45:00,основы программирования,потерял часа вебинаре,"понял, шла речь",проводили,безнадежность,0,0,0.0
2,2,2024-04-18 11:20:00,основы программирования,потеря времени,пропустил половину абсолютной нудятины,таких вебинаров следует избегать,полное разочарование,0,0,0.0
3,3,2024-04-19 16:55:00,основы программирования,просто ужас,запомнил,полное отсутствие смысла,просто кошмар,0,0,0.0
4,4,2024-04-20 13:10:00,основы программирования,никакой пользы извлек,потерял всякий интерес,поспал это время,ужасное впечатление,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
1310,999,2024-04-07 07:54:02,основы программирования,"заинтересовало, вебинаре обсуждался инструмент...","да, некоторые проблемы объяснением некоторых а...","кажется, стоит времени уделить рассмотрению те...",хотелось узнать библиотеках,1,1,0.0
1311,1000,2024-03-29 05:55:08,продвинутые техники программирования,вебинар проектировании оказался очень полезным,"согласен, примеры рефакторинга могли сложными ...",рабочие листы помогут усвоить материал,действительно узнал вебинара принципах проекти...,1,1,1.0
1312,1001,2024-04-24 20:25:25,управление персоналом,превосходно,расписание оказалось неудобным пересечения дру...,никаких нареканий,чат студентами оказался полезным дополнением,1,1,0.0
1313,1002,2024-04-22 19:02:00,новейшие тенденции,вряд заинтересовало вебинаре новейшие тенденци...,"возможно, вебинаре новейшие тенденции моменты,...","быть, вебинаре новейшие тенденции моменты, кот...","интересуют другие темы, связанные преподавател...",0,2,0.0


In [82]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 24
VALID_BATCH_SIZE = 48
# EPOCHS = 1
LEARNING_RATE = 1e-05

In [83]:
tokenizer = RobertaTokenizer.from_pretrained('ai-forever/ruRoberta-large', truncation=True, do_lower_case=True)

In [84]:
data_is_relevant = data.copy()
data_object = data.copy()
data_is_positive = data.copy()

# is_relevant

In [85]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['question_1'] + " " + dataframe['question_2'] + " " + dataframe['question_3'] + " " + dataframe['question_4'] + " " + dataframe['question_5']
        self.targets = self.data['is_relevant']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [86]:
train_size = 0.8
train_data=data_is_relevant.sample(frac=train_size,random_state=42)

test_data=data_is_relevant.drop(train_data.index)
test_data_index = test_data.index.tolist()
test_data=test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(data_is_relevant.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(data, tokenizer, MAX_LEN)
testing_set = SentimentData(data_eval, tokenizer, MAX_LEN)
all_data_set = SentimentData(data_is_relevant, tokenizer, MAX_LEN)

FULL Dataset: (1315, 10)
TRAIN Dataset: (1052, 10)
TEST Dataset: (263, 10)


In [87]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
all_df_params = {'batch_size': 24,
                'shuffle': False,
                'num_workers': 0
                }



training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
all_data_loader = DataLoader(all_data_set, **all_df_params)

In [88]:
class is_relevant(torch.nn.Module):
    def __init__(self):
        super(is_relevant, self).__init__()
        self.l1 = RobertaModel.from_pretrained('ai-forever/ruRoberta-large')
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(1024, 2) # 4 classes for category

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
    def get_embed(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        return pooler
    
model_is_relevant = is_relevant()
model_is_relevant.to(device)
print('ok')

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ok


In [89]:
loss_function = torch.nn.BCEWithLogitsLoss()
LEARNING_RATE = 1e-05
optimizer = torch.optim.Adam(params =  model_is_relevant.parameters(), lr=LEARNING_RATE)

In [90]:
epochs = 3
for epoch in range(0, epochs):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model_is_relevant.train()
    for _,data_train in tqdm(enumerate(all_data_loader, 0)):
        ids = data_train['ids'].to(device, dtype = torch.long)
        mask = data_train['mask'].to(device, dtype = torch.long)
        token_type_ids = data_train['token_type_ids'].to(device, dtype = torch.long)
        targets = data_train['targets'].to(device, dtype = torch.long)

        outputs = model_is_relevant(ids, mask, token_type_ids)
        num_classes = 2  # Два класса
        targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=num_classes)
        targets_one_hot = targets_one_hot.float()
        outputs = outputs.float()
        loss = loss_function(outputs, targets_one_hot)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1])
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss per 5000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        

            
    epoch_loss = tr_loss/nb_tr_steps     
    print(f"Epoch {epoch+1}/{epochs},Training Loss Epoch: {epoch_loss}") 

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training Loss per 5000 steps: 0.652361273765564


55it [00:39,  1.38it/s]
0it [00:00, ?it/s]

Epoch 1/3,Training Loss Epoch: 0.5007578985257582


1it [00:00,  1.43it/s]

...

Epoch 2/3,Training Loss Epoch: 0.6414019725539467


1it [00:00,  1.41it/s]

Training Loss per 5000 steps: 1.7998837232589722


55it [00:41,  1.33it/s]

Epoch 3/3,Training Loss Epoch: 0.6888151390986009


In [91]:
outputs_all = []
targets_all = []
with torch.no_grad():
    for _,data_res in tqdm(enumerate(testing_loader, 0)):
        ids = data_res['ids'].to(device, dtype = torch.long)
        mask = data_res['mask'].to(device, dtype = torch.long)
        token_type_ids = data_res['token_type_ids'].to(device, dtype=torch.long)
        targets = data_res['targets'].to(device, dtype = torch.long)
        embed = model_is_relevant.get_embed(ids, mask, token_type_ids)
        outputs_all += embed.cpu().numpy().tolist()
        targets_all += targets.cpu().numpy().tolist()

3it [00:01,  2.10it/s]


In [92]:
df_embed_is_relevant = pd.DataFrame(outputs_all)

In [93]:
df_embed_is_relevant

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.081664,0.279511,-1.306082,-0.048281,0.999499,-1.936790,-0.291874,0.627558,-0.192689,-0.232225,...,0.445116,-0.063830,0.138965,-0.343812,-0.585849,-0.177153,-0.014231,0.115830,0.626986,0.549741
1,1.460130,0.599713,-1.429861,-0.213349,0.330546,-1.774535,-0.550755,1.276812,0.100365,-0.643270,...,0.806015,-0.308411,0.400529,0.057245,-1.349308,-0.300856,-0.226646,0.112459,0.889113,0.837405
2,0.539640,-0.426453,-1.198697,-0.152789,0.434318,-2.296316,-0.424175,1.190953,-0.005965,-0.191491,...,0.402574,-0.946041,0.601490,0.302094,-0.450755,-0.036583,0.434536,0.961563,1.493554,0.235780
3,0.663000,0.160556,-0.733101,-0.633255,0.396868,-1.729588,0.071108,1.353120,0.927181,-0.845682,...,1.347812,0.147930,-0.462270,-0.871620,-0.836188,-0.135969,-0.004095,-0.184775,0.903543,0.374902
4,0.726380,0.488604,-1.728621,-0.304118,0.171412,-1.156725,-0.647366,1.139533,0.579269,-0.372429,...,0.105383,-0.158838,0.184433,-1.111090,-0.336094,-0.509820,-0.395262,-0.159298,1.061503,-0.367074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.676045,-0.336093,-1.516252,-0.549135,0.268364,-1.927011,-0.167268,1.328393,0.217344,-0.776154,...,0.629019,-0.515349,0.694408,-0.438487,-0.874299,-0.014681,-0.161659,0.378393,0.777127,0.517771
99,0.027436,0.422550,-1.127410,-0.094676,0.630936,-2.311038,-0.346903,1.028845,0.125721,0.189822,...,-0.556769,-0.058344,0.261299,-0.615340,-0.511161,0.098427,-0.211003,-0.131077,1.001691,0.318375
100,0.682844,1.225882,-1.169281,-0.170147,0.732660,-2.141634,0.395839,1.414442,0.330094,-0.839418,...,0.787654,-0.464076,-0.359345,-1.019804,-1.196739,-0.299736,-0.329388,-0.029063,1.027460,0.066824
101,1.273941,0.356838,-0.971498,-0.421066,-0.259383,-1.927647,-0.098157,0.972594,0.378320,-0.123138,...,0.499048,-0.133155,0.349798,-0.549505,-0.928472,-0.069889,-0.051815,0.646629,0.718018,0.014646


In [94]:
df_embed_is_relevant.to_csv('df_embed_is_relevant.csv')

# is_positive

In [95]:
class is_positiveData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['question_1'] + " " + dataframe['question_2'] + " " + dataframe['question_3'] + " " + dataframe['question_4'] + " " + dataframe['question_5']
        self.targets = self.data['is_positive']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [58]:
train_size = 0.8
train_data=data_is_positive.sample(frac=train_size,random_state=42)

test_data=data_is_positive.drop(train_data.index)
test_data_index = test_data.index.tolist()
test_data=test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(data_is_positive.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = is_positiveData(train_data, tokenizer, MAX_LEN)
testing_set = is_positiveData(test_data, tokenizer, MAX_LEN)
all_data_set = is_positiveData(data_is_positive, tokenizer, MAX_LEN)

FULL Dataset: (1315, 10)
TRAIN Dataset: (1052, 10)
TEST Dataset: (263, 10)


In [59]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
all_df_params = {'batch_size': 24,
                'shuffle': False,
                'num_workers': 0
                }



training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
all_data_loader = DataLoader(all_data_set, **all_df_params)

In [60]:
class is_positive(torch.nn.Module):
    def __init__(self):
        super(is_positive, self).__init__()
        self.l1 = RobertaModel.from_pretrained('ai-forever/ruRoberta-large')
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(1024, 2) # 4 classes for category

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
    def get_embed(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        return pooler
    
model_is_positive = is_positive()
model_is_positive.to(device)
print('ok')

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ok


In [61]:
loss_function = torch.nn.BCEWithLogitsLoss()
LEARNING_RATE = 1e-05
optimizer = torch.optim.Adam(params =  model_is_relevant.parameters(), lr=LEARNING_RATE)

In [62]:
epochs = 3
for epoch in range(0, epochs):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model_is_positive.train()
    for _,data_train in tqdm(enumerate(all_data_loader, 0)):
        ids = data_train['ids'].to(device, dtype = torch.long)
        mask = data_train['mask'].to(device, dtype = torch.long)
        token_type_ids = data_train['token_type_ids'].to(device, dtype = torch.long)
        targets = data_train['targets'].to(device, dtype = torch.long)

        outputs = model_is_positive(ids, mask, token_type_ids)
        num_classes = 2  # Два класса
        targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=num_classes)
        targets_one_hot = targets_one_hot.float()
        outputs = outputs.float()
        loss = loss_function(outputs, targets_one_hot)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1])
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss per 5000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        

            
    epoch_loss = tr_loss/nb_tr_steps     
    print(f"Epoch {epoch+1}/{epochs},Training Loss Epoch: {epoch_loss}") 

0it [00:00, ?it/s]

1it [00:00,  2.34it/s]

Training Loss per 5000 steps: 0.6803669929504395


55it [00:39,  1.39it/s]
0it [00:00, ?it/s]

Epoch 1/3,Training Loss Epoch: 0.7161724404855208
Training Loss per 5000 steps: 0.6916036605834961


55it [00:39,  1.38it/s]
0it [00:00, ?it/s]

Epoch 2/3,Training Loss Epoch: 0.7179316834969954
Training Loss per 5000 steps: 0.6880072951316833


55it [00:40,  1.37it/s]

Epoch 3/3,Training Loss Epoch: 0.7144191611896862


In [63]:
outputs_all = []
targets_all = []
with torch.no_grad():
    for _,data_res in tqdm(enumerate(all_data_loader, 0)):
        ids = data_res['ids'].to(device, dtype = torch.long)
        mask = data_res['mask'].to(device, dtype = torch.long)
        token_type_ids = data_res['token_type_ids'].to(device, dtype=torch.long)
        targets = data_res['targets'].to(device, dtype = torch.long)
        embed = model_is_positive.get_embed(ids, mask, token_type_ids)
        outputs_all += embed.cpu().numpy().tolist()
        targets_all += targets.cpu().numpy().tolist()

55it [00:15,  3.50it/s]


In [64]:
df_embed_is_positive = pd.DataFrame(outputs_all)

In [65]:
df_embed_is_positive.to_csv('df_embed_is_positive.csv')

# object

In [66]:
class objectData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['question_1'] + " " + dataframe['question_2'] + " " + dataframe['question_3'] + " " + dataframe['question_4'] + " " + dataframe['question_5']
        self.targets = self.data['object']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [67]:
train_size = 0.8
train_data=data_object.sample(frac=train_size,random_state=42)

test_data=data_object.drop(train_data.index)
test_data_index = test_data.index.tolist()
test_data=test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(data_object.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = objectData(train_data, tokenizer, MAX_LEN)
testing_set = objectData(test_data, tokenizer, MAX_LEN)
all_data_set = objectData(data_object, tokenizer, MAX_LEN)

FULL Dataset: (1315, 10)
TRAIN Dataset: (1052, 10)
TEST Dataset: (263, 10)


In [68]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
all_df_params = {'batch_size': 24,
                'shuffle': False,
                'num_workers': 0
                }



training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
all_data_loader = DataLoader(all_data_set, **all_df_params)

In [69]:
class object_m(torch.nn.Module):
    def __init__(self):
        super(object_m, self).__init__()
        self.l1 = RobertaModel.from_pretrained('ai-forever/ruRoberta-large')
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(1024, 3) # 4 classes for category

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
    def get_embed(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        return pooler
    
model_object = object_m()
model_object.to(device)
print('ok')

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ok


In [70]:
loss_function = torch.nn.BCEWithLogitsLoss()
LEARNING_RATE = 1e-05
optimizer = torch.optim.Adam(params =  model_is_relevant.parameters(), lr=LEARNING_RATE)

In [71]:
epochs = 3
for epoch in range(0, epochs):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model_object.train()
    for _,data_train in tqdm(enumerate(all_data_loader, 0)):
        ids = data_train['ids'].to(device, dtype = torch.long)
        mask = data_train['mask'].to(device, dtype = torch.long)
        token_type_ids = data_train['token_type_ids'].to(device, dtype = torch.long)
        targets = data_train['targets'].to(device, dtype = torch.long)

        outputs = model_object(ids, mask, token_type_ids)
        num_classes = 3  # Два класса
        targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=num_classes)
        targets_one_hot = targets_one_hot.float()
        outputs = outputs.float()
        loss = loss_function(outputs, targets_one_hot)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1,2])
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss per 5000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        

            
    epoch_loss = tr_loss/nb_tr_steps     
    print(f"Epoch {epoch+1}/{epochs},Training Loss Epoch: {epoch_loss}") 

0it [00:00, ?it/s]

1it [00:00,  2.33it/s]

Training Loss per 5000 steps: 0.6499131917953491


55it [00:39,  1.39it/s]
0it [00:00, ?it/s]

Epoch 1/3,Training Loss Epoch: 0.6971661307594993
Training Loss per 5000 steps: 0.6842942833900452


55it [00:39,  1.38it/s]
0it [00:00, ?it/s]

Epoch 2/3,Training Loss Epoch: 0.696126872842962
Training Loss per 5000 steps: 0.6698768138885498


55it [00:39,  1.38it/s]

Epoch 3/3,Training Loss Epoch: 0.6985671975395896


In [72]:
outputs_all = []
targets_all = []
with torch.no_grad():
    for _,data_res in tqdm(enumerate(all_data_loader, 0)):
        ids = data_res['ids'].to(device, dtype = torch.long)
        mask = data_res['mask'].to(device, dtype = torch.long)
        token_type_ids = data_res['token_type_ids'].to(device, dtype=torch.long)
        targets = data_res['targets'].to(device, dtype = torch.long)
        embed = model_object.get_embed(ids, mask, token_type_ids)
        outputs_all += embed.cpu().numpy().tolist()
        targets_all += targets.cpu().numpy().tolist()

55it [00:15,  3.52it/s]


In [73]:
df_embed_object = pd.DataFrame(outputs_all)

In [74]:
df_embed_object.to_csv('df_embed_object.csv')